In [1]:
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from functools import reduce
from IPython.core.display import HTML
import webbrowser
from datetime import datetime

In [2]:
#Open all correct dataframes
btc_1d = pd.read_csv('BFX-BTCUSD-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_1h = pd.read_csv('BFX-BTCUSD-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_15m = pd.read_csv('BFX-BTCUSD-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
btc_1m = pd.read_csv('BFX-BTCUSD-1m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

eth_1d = pd.read_csv('BFX-ETHUSD-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
eth_1h = pd.read_csv('BFX-ETHUSD-1H.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
eth_15m = pd.read_csv('BFX-ETHUSD-15M.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

busd_1d = pd.read_csv('BNC-BTCBUSD-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
busd_1h = pd.read_csv('BNC-BTCBUSD-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
busd_15m = pd.read_csv('BNC-BTCBUSD-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

usdt_1d = pd.read_csv('BNC-BTCUSDT-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdt_1h = pd.read_csv('BNC-BTCUSDT-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdt_15m = pd.read_csv('BNC-BTCUSDT-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

usdc_1d = pd.read_csv('BNC-BTCUSDC-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdc_1h = pd.read_csv('BNC-BTCUSDC-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
usdc_15m = pd.read_csv('BNC-BTCUSDC-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

dai_1d = pd.read_csv('BNC-BTCDAI-1d.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
dai_1h = pd.read_csv('BNC-BTCDAI-1h.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])
dai_15m = pd.read_csv('BNC-BTCDAI-15m.csv', usecols=['timestamp', 'open', 'close', 'high', 'low', 'volume'])

In [3]:
def preparing_DAI_data(df1, df2, df3, df4, df5, tokenA, tokenB, tokenC, tokenD, tokenE):
    #df1 is BTC
    df1_copy = df1[['timestamp', 'close', 'volume']].copy()
    #df2 is USDT
    df2_copy = df2[['timestamp', 'close', 'volume']].copy()
    #df3 is DAI, df4 USDC, df5 is ETH
    df3_copy = df3[['timestamp', 'close', 'volume']].copy()
    df4_copy = df4[['timestamp', 'close', 'volume']].copy()
    df5_copy = df5[['timestamp', 'close', 'volume']].copy()
    
    #normalize stables against USD
    stables = [[df3_copy, tokenC], [df4_copy, tokenD]]
    for stable in stables:
        temp_stable = df2_copy.merge(stable[0], how='left', on=['timestamp'])
        temp_stable['close'] = temp_stable['close_y'] * temp_stable['close_x']
        temp_stable['volume'] = temp_stable['volume_y']
        stable[0] = temp_stable[['timestamp', 'close', 'volume']].copy()
    
    #Calculate the return on all tokens
    lst = [[df1_copy, tokenA], [df2_copy, tokenB], [df3_copy, tokenC], [df4_copy, tokenD], [df5_copy, tokenE]]
    for pair in lst:
        column_name = 'return_' + pair[1]
        pair[0].columns = ['timestamp', 'close_' + pair[1], 'volume_' + pair[1]]
        pair[0][column_name] = (pair[0]['close_' + pair[1]].pct_change())
        pair[0][column_name + '_log'] = (np.log(1 + pair[0][column_name])) * 100
    
    # Merge all dataframes
    merged = df1_copy.copy()
    lst2 = [df3_copy, df4_copy, df5_copy]
    for df in lst2:
        merged = merged.merge(df, how='left', on=['timestamp'])
        
    merged['portfolio_return'] = 0.059 * merged['return_BTC'] + 0.389 * merged['return_ETH'] + 0.562 * merged['return_USDC']
    merged['portfolio_return_log'] = (np.log(1 + merged['portfolio_return'])) * 100
    
    merged['Dq_10'] = merged['portfolio_return_log'].apply(lambda x: 1 if x < merged['portfolio_return_log'].quantile(.1) else 0)
    merged['Dq_05'] = merged['portfolio_return_log'].apply(lambda x: 1 if x < merged['portfolio_return_log'].quantile(.05) else 0)
    merged['Dq_01'] = merged['portfolio_return_log'].apply(lambda x: 1 if x < merged['portfolio_return_log'].quantile(.01) else 0)
    
    merged.set_index('timestamp', inplace=True)
        
    return merged



def transform_and_merge(df1, df2, df3, df4, tokenA, tokenB, tokenC, tokenD):
    #use only relevant columns
    #df1 is BTC/USD
    df1_copy = df1[['timestamp', 'close', 'volume']].copy()
    df2_copy = df2[['timestamp', 'close', 'volume']].copy()
    df3_copy = df3[['timestamp', 'close', 'volume']].copy()
    df4_copy = df4[['timestamp', 'close', 'volume']].copy()
#     df5_copy = df5[['timestamp', 'close', 'volume']].copy()
    
    # inverse tokens
    stables = [[df1_copy, tokenA], [df2_copy, tokenB], [df3_copy, tokenC], [df4_copy, tokenD]]
    for stable in stables:
        stable[0]['close'] = 1/stable[0]['close']
    
    # Calculate log returns for all tokens + rename all columns
    lst = [[df1_copy, tokenA], [df2_copy, tokenB], [df3_copy, tokenC], [df4_copy, tokenD],]
    for pair in lst:
        column_name = 'return_' + pair[1]
        pair[0].columns = ['timestamp', 'close_' + pair[1], 'volume_' + pair[1]]
        pair[0][column_name] = (pair[0]['close_' + pair[1]].pct_change())
        pair[0][column_name + '_log'] = (np.log(1 + pair[0][column_name])) * 100
        
    # Make columns for every quantile relevant for BTC
    df1_copy['Dq_10'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.1) else 0)
    df1_copy['Dq_05'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.05) else 0)
    df1_copy['Dq_01'] = df1_copy['return_BTC_log'].apply(lambda x: 1 if x < df1_copy['return_BTC_log'].quantile(.01) else 0)

    # Merge all dataframes
    merged = df1_copy.copy()
    lst2 = [df2_copy, df3_copy, df4_copy]
    for df in lst2:
        merged = merged.merge(df, how='left', on=['timestamp'])
    merged.set_index('timestamp', inplace=True)  
    return merged



In [6]:
# 15-minute timeframe
dataframe_dai = preparing_DAI_data(btc_15m, usdt_15m, dai_15m, usdc_15m, eth_15m, 'BTC','USDT', 'DAI', 'USDC', 'ETH')
dataframe_stables = transform_and_merge(btc_15m, usdc_15m, usdt_15m, busd_15m, 'BTC','USDC', 'USDT', 'BUSD')

est = smf.ols(formula='return_USDT_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est3 = smf.ols(formula='return_BUSD_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est2 = smf.ols(formula='return_USDC_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est4 = smf.ols(formula='return_DAI_log ~  portfolio_return_log * Dq_10 + portfolio_return_log * Dq_05 + portfolio_return_log * Dq_01', data=dataframe_dai).fit(cov_type='HAC', cov_kwds={'maxlags':1})

stargazer = Stargazer([est, est2, est3, est4])

stargazer.custom_columns(['USDT', 'USDC', 'BUSD', 'DAI'], [1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.title(f'Collateral Test: Regression results of the 15-minute timeframe')
stargazer

In [9]:
# 1-hour timeframe
dataframe_dai = preparing_DAI_data(btc_1h, usdt_1h, dai_1h, usdc_1h, eth_1h, 'BTC','USDT', 'DAI', 'USDC', 'ETH')
dataframe_stables = transform_and_merge(btc_1h, usdc_1h, usdt_1h, busd_1h, 'BTC','USDC', 'USDT', 'BUSD')

est = smf.ols(formula='return_USDT_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est3 = smf.ols(formula='return_BUSD_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est2 = smf.ols(formula='return_USDC_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est4 = smf.ols(formula='return_DAI_log ~  portfolio_return_log * Dq_10 + portfolio_return_log * Dq_05 + portfolio_return_log * Dq_01', data=dataframe_dai).fit(cov_type='HAC', cov_kwds={'maxlags':1})

stargazer = Stargazer([est, est2, est3, est4])

stargazer.custom_columns(['USDT', 'USDC', 'BUSD', 'DAI'], [1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.title(f'Collateral Test: Regression results of the 1-hour timeframe')
stargazer

In [ ]:
# daily timeframe
dataframe_dai = preparing_DAI_data(btc_1d, usdt_1d, dai_1d, usdc_1d, eth_1d, 'BTC','USDT', 'DAI', 'USDC', 'ETH')
dataframe_stables = transform_and_merge(btc_1d, usdc_1d, usdt_1d, busd_1d, 'BTC','USDC', 'USDT', 'BUSD')

est = smf.ols(formula='return_USDT_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est3 = smf.ols(formula='return_BUSD_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est2 = smf.ols(formula='return_USDC_log ~  return_BTC_log * Dq_10 + return_BTC_log * Dq_05 + return_BTC_log * Dq_01', data=dataframe_stables).fit(cov_type='HAC', cov_kwds={'maxlags':1})
est4 = smf.ols(formula='return_DAI_log ~  portfolio_return_log * Dq_10 + portfolio_return_log * Dq_05 + portfolio_return_log * Dq_01', data=dataframe_dai).fit(cov_type='HAC', cov_kwds={'maxlags':1})

stargazer = Stargazer([est, est2, est3, est4])

stargazer.custom_columns(['USDT', 'USDC', 'BUSD', 'DAI'], [1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.title(f'Collateral Test: Regression results of the daily timeframe')
stargazer